In [1]:
import gradio as gr
import pandas as pd
import numpy as np
import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from bayes_opt import BayesianOptimization

MODEL_PATH = "models/gbm_model.pkl"  

def process_and_predict(file2, file3):
    sheet_name = "実績昼"
    def clean_data(file_path, sheet_name):

        # データ読み込み
        df = pd.read_excel(file_path, sheet_name=sheet_name)
    
        # 転置
        df = df.T

        # 1行目をカラム名にして削除
        df.columns = df.iloc[1]
        df = df.drop(df.index[[0, 1]])

        # 「項目」列がある2列目以降を抽出（1列目は"作業ライン"、2列目が"項目"）
        df_section = df.iloc[1:10, 1:21]  # 2行目〜10行目、2列目〜21列目（日付1〜20）

        # 最初の列をインデックス（項目名）として、列を日付とするように転置
        df_section.columns = df_section.iloc[0]  # 2行目（"件数"など）をカラム名にする
        df_section = df_section.drop(df_section.index[0])  # カラム名に使った行を削除

        # 行番号を日付（1〜20）にして分かりやすく
        df_section = df_section.reset_index(drop=True)
        df_section.insert(0, "日付", range(1, len(df_section) + 1))


        # 削除対象のカラム名
        columns_to_drop = [
            np.nan,
            'テイケイ',
            '目標件数',
            '予測',
            '1本あたりの平均作業数',
            '1本あたりの平均作業', 
            '1日あたりの作業件数',
            '買い合わせ平均',      
            '項目',
            '生産性(分/件)',
            '1日作業量/L'
        ]

        # 一致するカラムのみ削除
        df = df.drop(columns=[col for col in df.columns if col in columns_to_drop])

        # 欠損値を0で埋める
        df = df.fillna(0)

        # 34行目（インデックス33）を削除
        df.drop(df.index[[31]])

        # 「件数」が0の行を削除（数値変換してから）
        if '1人あたりの作業時間' in df.columns:
            df['1人あたりの作業時間'] = pd.to_numeric(df['1人あたりの作業時間'], errors='coerce').fillna(0)
            df = df[df['1人あたりの作業時間'] != 0]

        return df


    df2 = clean_data(file2, sheet_name )
    df3 = clean_data(file3, sheet_name)

    # 対数変換
    df2 = df2.apply(lambda x: np.log(x + 1))
    df3 = df3.apply(lambda x: np.log(x + 1))

    # カラム順合わせ
    df2 = df2[df3.columns]

    # 学習・予測用データ
    X_train = df2.drop(columns='件数')
    y_train = df2['件数']
    X_test = df3.drop(columns='件数')
    y_test = df3['件数']

    # ベイズ最適化
    def optimize_model(n_estimators, learning_rate, max_depth, max_features, min_samples_split, subsample):
        model = GradientBoostingRegressor(
            n_estimators=int(n_estimators),
            learning_rate=learning_rate,
            max_depth=int(max_depth),
            max_features=max_features,
            min_samples_split=int(min_samples_split),
            subsample=subsample,
            random_state=42
        )
        return cross_val_score(model, X_train, y_train, cv=5, scoring='r2').mean()

    pbounds = {
        'n_estimators': (100, 1000),
        'learning_rate': (0.005, 0.3),
        'max_depth': (3, 15),
        'min_samples_split': (2, 20),
        'subsample': (0.5, 1.0),
        'max_features': (0.5, 1.0)
    }

    optimizer = BayesianOptimization(f=optimize_model, pbounds=pbounds, random_state=42, verbose=0)
    optimizer.maximize(init_points=5, n_iter=20)
    best_params = optimizer.max['params']
    best_params['n_estimators'] = int(best_params['n_estimators'])
    best_params['max_depth'] = int(best_params['max_depth'])
    best_params['min_samples_split'] = int(best_params['min_samples_split'])

    model = GradientBoostingRegressor(**best_params, random_state=42)
    model.fit(X_train, y_train)

    # 予測と元スケールへの逆変換
    y_pred = model.predict(X_test)
    y_pred_original = np.exp(y_pred)
    y_test_original = np.exp(y_test)

    # 'Unnamed:' を削除し、数値を-1する
    modified_index = [int(item.replace('Unnamed: ', '')) - 1 for item in df3.index]

    result_df = pd.DataFrame({
        "日付": modified_index,
        "予測件数": y_pred_original,
        "実績件数": y_test_original.values,
        "差分（予測件数ー実績件数）": y_pred_original - y_test_original.values
    })
    # ① to_html でクラス名を付与して HTML テーブル文字列を生成
    html_table = result_df.to_html(
        classes="result-table", 
        index=False, 
        border=1
    )

    r2 = r2_score(y_test_original, y_pred_original)
    summary = f"R²スコア: {r2:.3f}（1に近いほど良好）"

    return summary, html_table

# --- CSS を定義 ---
custom_css = """
.result-table {
  border-collapse: collapse;
  width: 100%;
}
.result-table th,
.result-table td {
  border: 1px solid #ddd;
  padding: 8px;
  text-align: center;
}
/* 「予測件数」列だけ赤文字に */
.result-table td:nth-child(2),
.result-table th:nth-child(2) {
  color: red;
  font-weight: bold;
}
"""


# --- Gradio UI ---
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("## 平和島LBの作業件数予測")
    gr.Markdown("過去実績をもとに出荷月の作業件数を予測します")
    with gr.Row():
        with gr.Column():
            inp1 = gr.File(label="2月のExcelファイル（件数あり）")
            inp2 = gr.File(label="出荷月のExcelファイル")
            btn = gr.Button("予測実行")
        with gr.Column():
            out1 = gr.Text(label="予測モデルの性能")
            out2 = gr.HTML(label="各日の予測 vs 実績（件数）")
    btn.click(
        fn=process_and_predict,
        inputs=[inp1, inp2],
        outputs=[out1, out2],
    )

demo.launch()

/Users/yasudayuuya/orderpredict/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/var/folders/wl/_sk4lq9107v79smv_pqvls780000gn/T/ipykernel_76122/596951432.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/var/folders/wl/_sk4lq9107v79smv_pqvls780000gn/T/ipykernel_76122/596951432.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
/var/folders/wl/_sk4lq9107v79smv_pqvls780000gn/T/ipykernel_76122/596951432.py:57: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to t